In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
num_age_classes = 100
batch_size = 1024

In [3]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
train_dataset = ImageFolder('/Users/arpan/Desktop/ml-unlearning/dataset/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [5]:
import torch.nn as nn

class AgePredictionCNN(nn.Module):
    def __init__(self, num_classes):
        super(AgePredictionCNN, self).__init__()
        self.features = nn.Sequential(
            # Define convolutional layers here
            # Example:
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            # Add more layers as needed
        )
        self.fc = nn.Sequential(
            # Define fully connected layers here
            # Example:
            nn.Linear(64 * 112 * 112, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)  # Output layer for age prediction
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Create an instance of the model
model = AgePredictionCNN(num_classes=num_age_classes)


In [6]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float()) 
        loss.backward()
        optimizer.step()


  0%|          | 0/182 [00:00<?, ?it/s]

/Users/arpan/Desktop/ml-unlearning/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1024, 1])) that is different to the input size (torch.Size([1024, 100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  4%|▍         | 8/182 [10:24<3:46:43, 78.18s/it]

In [ ]:
test_dataset = ImageFolder('/Users/arpan/Desktop/ml-unlearning/dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval() 

total_loss = 0.0 
total_samples = 0 

for inputs, labels in test_loader: 
    with torch.no_grad(): 
        outputs = model(inputs)
        labels = labels.float()

        loss = criterion(outputs, labels.unsqueeze(1))

        total_loss += loss.item()
        total_samples += labels.size(0)

average_loss = total_loss / len(validation_loader)  

print(f"Validation/Test Loss: {average_loss:.4f}")


mae = torch.abs(outputs - labels).mean()
print(f"Mean Absolute Error (MAE): {mae:.2f}")

rmse = math.sqrt(((outputs - labels) ** 2).mean())
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")